In [1]:
from math import sqrt, log, log10, log2
from operator import itemgetter

import numpy as np
from scipy.linalg import svd
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
from sklearn import metrics


In [92]:
class BPR:
    def __init__(self, records_train, records_test):
        records = np.vstack([records_train, records_test])

        self.n = records[:, 0].max() + 1
        self.m = records[:, 1].max() + 1
        
        # Initial R
        self.R = np.zeros([self.n, self.m], dtype=bool)
        for record in records_train:
            if record[2] < 4:
                continue
            self.R[record[0], record[1]] = True

        # Initial R_test
        self.R_test = np.zeros([self.n, self.m], dtype=bool)
        for record in records_test:
            if record[2] < 4:
                continue
            self.R_test[record[0], record[1]] = True
        
        # Initial indicator
        y = np.where(self.R, 1, 0)
        self.y_user = np.sum(y, axis=1)
        y_item = np.sum(y, axis=0)
        
        y_test = np.where(self.R_test, 1, 0)
        self.y_user_test = np.sum(y_test, axis=1)
        y_item_test = np.sum(y_test, axis=0)

        # Global average of rating
        self.mu = np.sum(y) / self.n / self.m

        # bias of item
        self.b_i = np.where(y_item,
                            y_item / self.n - self.mu,
                            0)
     
        self.users_test = np.nonzero(self.y_user_test)[0]
        
        
    def gradient_descent(self, n_iter=50):

        alpha = 0.01
        d = 20
        
        # Initialize
        self.U = (np.random.randint(0, 1, size=(self.n, d)) - 0.5) * 0.01
        self.V = (np.random.randint(0, 1, size=(self.m, d)) - 0.5) * 0.01
        
        
        eta = 0.05
        
        def sigmoid(x):
            return 1.0 / (1 + np.exp(-x))
        
        
        def update(user, item_i, item_j):
            # update
            r_uij = self.predict(user, item_i) - self.predict(user, item_j)
            e = -sigmoid(-r_uij)
            d_U = e * (self.V[item_i] - self.V[item_j]) + alpha * self.U[user]
            d_V_i = e * self.U[user] + alpha * self.V[item_i]
            d_V_j = e * (-self.U[user]) + alpha * self.V[item_j]
            d_b_i = e + alpha * self.b_i[item_i]
            d_b_j = -e + alpha * self.b_i[item_j]
            
            self.U[user, :] -= eta * d_U
            self.V[item_i, :] -= eta * d_V_i
            self.V[item_j, :] -= eta * d_V_j
            self.b_i[item_i] -= eta * d_b_i
            self.b_i[item_j] -= eta * d_b_j
            

        for cur_iter in range(n_iter):
            print(cur_iter)
            ratings = np.where(self.R != 0)
            num = len(ratings[0])
            indexes = np.random.permutation(num)
            users = ratings[0][indexes]
            items = ratings[1][indexes]

            for i in range(num):
                user = users[i]
                item_i = items[i]
                items_j = np.where(self.R[user, :] == 0)[0]
                item_j =  items_j[np.random.randint(0, len(items_j))]
                update(user, item_i, item_j)
                
            # eta = eta * 0.9
            
            # ratings_predict = self.performance()
            self.performance()
                

        return

    def predict(self, user, item):
        return self.U[user].dot(self.V[item]) + self.b_i[item]
                
    
    def performance(self):
        ratings_predict = self.U.dot(self.V.T) + self.b_i
        # i_rec = sorted(items, key=lambda x: ratings_predict[x], reverse=True)
        self.i_rec = np.zeros([self.n, 5], dtype=np.int32)
        for user in range(self.n):
            ratings_predict[user, np.where(self.R[user, :] != 0)[0]] = -10
            self.i_rec[user] = np.argsort(ratings_predict[user])[::-1][:5]
        self.get_pre(self.i_rec)
        # self.get_rec(i_rec)
    
    def get_pre(self, i_rec):
        self.pre = 0
        self.pre_u = np.zeros(self.n)
        for user in self.users_test:
            self.pre_u[user] = self.R_test[user, i_rec[user]].sum() / 5
            self.pre += self.R_test[user, i_rec[user]].sum() / 5
        self.pre /= len(self.users_test)
        print(round(self.pre, 4))
    
    def get_rec(self, i_rec):
        self.rec = 0
        self.rec_u = np.zeros(self.n)
        for user in self.users_test:
            self.rec_u[user] = self.R_test[user, i_rec[user]].sum() / self.y_user_test[user].sum()
            self.rec += self.R_test[user, i_rec[user]].sum() / self.y_user_test[user].sum()
        self.rec /= len(self.users_test)
        print(round(self.rec, 4))

In [93]:
# Load the records
records_train = np.loadtxt('../data/ml-100k/u1.base', dtype=np.int32)
records_test = np.loadtxt('../data/ml-100k/u1.test', dtype=np.int32)

# Preprocess
records_train[:, :2] -= 1
records_test[:, :2] -= 1


In [94]:
bpr = BPR(records_train, records_test)

In [95]:
bpr.gradient_descent()

0
0.2456
1
0.2263
2
0.2386
3
0.2417
4
0.239
5
0.2487
6
0.243
7
0.2487
8
0.2509
9
0.2425
10
0.2553
11
0.2693
12
0.2614
13
0.2706
14
0.2785
15
0.2741
16
0.2789
17
0.2781
18
0.2807
19
0.2645
20
0.2702
21
0.2772
22
0.2768
23
0.2719
24
0.2772
25
0.2689
26
0.2588
27
0.2851
28


KeyboardInterrupt: 

In [28]:
tt = np.array([1, 2, 3, 4, 5, 6])
tt[2:4]

array([3])